In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import seaborn as sns 
import matplotlib.pylab as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as path_effects
import json
import pingouin as pg

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


In [ ]:
filter = "rare"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]



In [ ]:
SEs["SIDER"].shape

In [ ]:
SEs["OFFSIDES"].shape

In [ ]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]

In [ ]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    # "RF":[C], 
    # "BRF":[C]
}

In [ ]:
SEs_name = "SIDER"
metrice = "AUPR"

In [ ]:
hyperpars = {}
hyperpars["nested_cv"] = {}
hyperpars["cv"] = {}
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

In [ ]:
# hyperpars["nested_cv"]["SKR"] = {}
# hyperpars["nested_cv"]["SKR"]["target"] = [
#     (0.01, 0.4, 10, 100),
#     (0.01, 0.4, 10, 100),
#     (0.01, 0.4, 10, 100),
#     (0.01, 0.5, 10, 100),
#     (0.01, 0.4, 10, 100),
#     ]
# hyperpars["nested_cv"]["SKR"]["enzyme"] = [
#     (0.01, 0.6, 10, 100),
#     (1, 0.8, 10, 100),
#     (1, 0.2, 10, 100),
#     (1, 0.2, 10, 100),
#     (1, 0.2, 10, 100),
#     ]
# hyperpars["nested_cv"]["SKR"]["Chem"] = [
#     (1, 0.3, 10, 100),
#     (1, 0.3, 10, 100),
#     (1, 0.3, 10, 100),
#     (1, 0.3, 10, 100),
#     (1, 0.3, 10, 100),
#     ]
# hyperpars["nested_cv"]["SKR"]["DGI"] = [
#     (0.1, 0.2, 10, 100),
#     (0.1, 0.2, 10, 100),
#     (0.1, 0.2, 10, 100),
#     (0.1, 0.1, 10, 100),
#     (0.1, 0.2, 10, 100),
#     ]
# # hyperpars["nested_cv"]["SKR"]["transporter"] = [
# #     (1, 0.3, 10, 1),
# #     (1, 0.1, 10, 100),
# #     (0.1, 0.5, 10, 100),
# #     (0.1, 0.5, 10, 100),
# #     (0.1, 0.5, 10, 100),
# #     ]
# hyperpars["nested_cv"]["SKR"]["transporter"] = [
#     (1, 0.1, 10, 100),
#     (1, 0.1, 10, 100),
#     (0.1, 0.5, 10, 100),
#     (0.1, 0.5, 10, 100),
#     (0.1, 0.5, 10, 100),
#     ]
# hyperpars["nested_cv"]["SKR"]["pathway"] = [
#     (0.01, 0.3, 10, 100),
#     (0.01, 0.3, 10, 100),
#     (0.01, 0.3, 10, 100),
#     (0.01, 0.3, 10, 100),
#     (0.01, 0.3, 10, 100),
#     ]
# hyperpars["nested_cv"]["SKR"]["indication"] = [
#     (0.1, 0.1, 10, 100),
#     (0.1, 0.1, 10, 100),
#     (0.1, 0.1, 10, 100),
#     (0.1, 0.1, 10, 100),
#     (0.1, 0.1, 10, 100),
#     ]
# # hyperpars["cv"]["SKR"] = {}
# # hyperpars["cv"]["SKR"]["target"] = (10, 0.9, 10, 100)
# # hyperpars["cv"]["SKR"]["enzyme"] = (1, 0.9, 10, 100)
# # hyperpars["cv"]["SKR"]["Chem"] = (0.01, 0.7, 100, 100)
# # hyperpars["cv"]["SKR"]["DGI"] =(0.1, 0.4, 100, 100)
# # hyperpars["cv"]["SKR"]["transporter"] = (10, 0.9, 10, 10)
# # hyperpars["cv"]["SKR"]["pathway"] = (0.1, 0.4, 100, 10)
# # hyperpars["cv"]["SKR"]["indication"] = (1, 0.4, 100, 10)
# hyperpars["cv"]["SKR"] = {}
# hyperpars["cv"]["SKR"]["target"] = (10, 0.9, 10, 100)
# hyperpars["cv"]["SKR"]["enzyme"] = (1, 0.9, 10, 100)
# hyperpars["cv"]["SKR"]["Chem"] = (1, 0.9, 10, 100)
# hyperpars["cv"]["SKR"]["DGI"] =(1, 0.9, 10, 100)
# hyperpars["cv"]["SKR"]["transporter"] = (100, 0.9, 10, 100)
# hyperpars["cv"]["SKR"]["pathway"] = (100, 0.5, 10, 100)
# hyperpars["cv"]["SKR"]["indication"] = (1, 0.9, 10, 100)

In [ ]:
# hyperpars["nested_cv"]["KRR"] = {}
# hyperpars["nested_cv"]["KRR"]["target"] = [
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["enzyme"] = [
#     (1, 10),
#     (10, 10),
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["Chem"] = [
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["DGI"] = [
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["transporter"] = [
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     (1, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["pathway"] = [
#     (0.01, 10),
#     (0.01, 10),
#     (0.01, 10),
#     (0.01, 10),
#     (0.01, 10),
#     ]
# hyperpars["nested_cv"]["KRR"]["indication"] = [
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     (0.1, 10),
#     ]
# hyperpars["cv"]["KRR"] = {}
# hyperpars["cv"]["KRR"]["target"] = (100, 10)
# hyperpars["cv"]["KRR"]["enzyme"] = (100, 10)
# hyperpars["cv"]["KRR"]["Chem"] = (0.1, 100)
# hyperpars["cv"]["KRR"]["DGI"] = (0.1, 100)
# hyperpars["cv"]["KRR"]["transporter"] = (100, 100)
# hyperpars["cv"]["KRR"]["pathway"] = (0.1, 100)
# hyperpars["cv"]["KRR"]["indication"] = (100, 100)

In [ ]:
# hyperpars["nested_cv"]["Naive"] = {}
# hyperpars["nested_cv"]["Naive"]["target"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["enzyme"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["Chem"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["DGI"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["transporter"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["pathway"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["nested_cv"]["Naive"]["indication"] = [
#     (),
#     (),
#     (),
#     (),
#     (),
#     ]
# hyperpars["cv"]["Naive"] = {}
# hyperpars["cv"]["Naive"]["target"] = ()
# hyperpars["cv"]["Naive"]["enzyme"] = ()
# hyperpars["cv"]["Naive"]["Chem"] =()
# hyperpars["cv"]["Naive"]["DGI"] = ()
# hyperpars["cv"]["Naive"]["transporter"] = ()
# hyperpars["cv"]["Naive"]["pathway"] = ()
# hyperpars["cv"]["Naive"]["indication"] = ()

In [ ]:
# hyperpars["nested_cv"]["VKR"] = {}
# hyperpars["nested_cv"]["VKR"]["target"] = [
#     (10, 0.1, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["enzyme"] = [
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["Chem"] = [
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["DGI"] = [
#     (10, 0.1, 15),
#     (10, 0.1, 10),
#     (10, 0.1, 15),
#     (10, 0.01, 15),
#     (10, 0.1, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["transporter"] = [
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 1, 15),
#     (10, 0.1, 15),
#     (10, 0.1, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["pathway"] = [
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     ]
# hyperpars["nested_cv"]["VKR"]["indication"] = [
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     (10, 0.01, 15),
#     ]
# hyperpars["cv"]["VKR"] = {}
# hyperpars["cv"]["VKR"]["target"] = (10, 100, 10)
# hyperpars["cv"]["VKR"]["enzyme"] = (10, 100, 10)
# hyperpars["cv"]["VKR"]["Chem"] =(100, 0.01, 10)
# hyperpars["cv"]["VKR"]["DGI"] = (100, 0.1, 10)
# hyperpars["cv"]["VKR"]["transporter"] = (100, 0.1, 15)
# hyperpars["cv"]["VKR"]["pathway"] = (100, 10, 15)
# hyperpars["cv"]["VKR"]["indication"] = (100, 1, 15)

In [ ]:
# hyperpars["nested_cv"]["LNSM_RLN"] = {}
# hyperpars["nested_cv"]["LNSM_RLN"]["target"] = [
#     (0.2, 10), 
#     (0.2, 10), 
#     (0.2, 10), 
#     (0.2, 10), 
#     (0.2, 10)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["enzyme"] = [
#     (0.1, 100),
#     (0.1, 100),
#     (0.1, 100),
#     (0.1, 100),
#     (0.1, 10)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["Chem"] = [
#     (0.6, 100),
#     (0.1, 100),
#     (0.6, 100),
#     (0.1, 100),
#     (0.6, 100)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["DGI"] = [
#     (0.2, 10),
#     (0.2, 10),
#     (0.3, 100),
#     (0.2, 10),
#     (0.2, 10)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["transporter"] = [
#     (0.3, 100),
#     (0.3, 10),
#     (0.3, 10),
#     (0.2, 10),
#     (0.3, 10)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["pathway"] = [
#     (0.1, 10), 
#     (0.1, 10), 
#     (0.1, 10), 
#     (0.1, 1), 
#     (0.1, 10)
#     ]
# hyperpars["nested_cv"]["LNSM_RLN"]["indication"] = [
#     (0.5, 100),
#     (0.4, 100),
#     (0.4, 100),
#     (0.3, 100),
#     (0.4, 100)
#     ]
# hyperpars["cv"]["LNSM_RLN"] = {}
# hyperpars["cv"]["LNSM_RLN"]["target"] = (0.1, 1)
# hyperpars["cv"]["LNSM_RLN"]["enzyme"] = (0.1, 100)
# hyperpars["cv"]["LNSM_RLN"]["Chem"] = (0.9, 100)
# hyperpars["cv"]["LNSM_RLN"]["DGI"] = (0.2, 10)
# hyperpars["cv"]["LNSM_RLN"]["transporter"] = (0.1, 10)
# hyperpars["cv"]["LNSM_RLN"]["pathway"] = (0.1, 10)
# hyperpars["cv"]["LNSM_RLN"]["indication"] = (0.3, 100)

In [ ]:
# hyperpars["nested_cv"]["LNSM_jaccard"] = {}
# hyperpars["nested_cv"]["LNSM_jaccard"]["target"] = [
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["enzyme"] = [
#     (0.2,), 
#     (0.2,), 
#     (0.2,), 
#     (0.2,), 
#     (0.2,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["Chem"] = [
#     (0.2,), 
#     (0.2,), 
#     (0.2,), 
#     (0.8,), 
#     (0.5,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["DGI"] = [
#     (0.2,), 
#     (0.2,), 
#     (0.2,), 
#     (0.2,), 
#     (0.2,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["transporter"] = [
#     (0.6,),
#     (0.7,),
#     (0.1,),
#     (0.7,),
#     (0.1,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["pathway"] = [
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,)
#     ]
# hyperpars["nested_cv"]["LNSM_jaccard"]["indication"] = [
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,), 
#     (0.1,)
#     ]
# hyperpars["cv"]["LNSM_jaccard"] = {}
# hyperpars["cv"]["LNSM_jaccard"]["target"] = (0.1,)
# hyperpars["cv"]["LNSM_jaccard"]["enzyme"] = (0.4,)
# hyperpars["cv"]["LNSM_jaccard"]["Chem"] = (0.4,)
# hyperpars["cv"]["LNSM_jaccard"]["DGI"] = (0.1,)
# hyperpars["cv"]["LNSM_jaccard"]["transporter"] = (0.5,)
# hyperpars["cv"]["LNSM_jaccard"]["pathway"] = (0.1,)
# hyperpars["cv"]["LNSM_jaccard"]["indication"] = (0.1,)

In [ ]:

# Open and read the JSON file
with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
    hyperpars = json.load(xml_file)

In [ ]:
for method in methods:
    # # method = "SKR"
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    # method = "SKR"
    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

In [ ]:
# with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
#    json.dump(hyperparsOut, xml_file)
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

In [45]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

In [46]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            0.835042   0.015121          0.916771   
            KRR            0.792077   0.021675          0.875733   
            VKR            0.799172   0.016793          0.804518   
            Naive          0.791896   0.011867          0.813665   
            LNSM_RLN       0.751680   0.151963          0.815806   
            LNSM_jaccard   0.606089   0.076855          0.666055   
Chem        SKR            0.799193   0.034896          0.857009   
            KRR            0.767846   0.035998          0.828240   
            VKR            0.770189   0.011737          0.785314   
            Naive          0.795526   0.006659          0.816770   
            LNSM_RLN       0.626013   0.054841          0.683727   
            LNSM_jaccard   0.512384   0.019665          0.521511   
DGI         SKR            0.783661   0.013205          0.849980   
            KRR            0.769053   0.013637          0.834155   
            VKR            0.773871   0.021402          0.772966   
            Naive          0.785489   0.005799          0.804576   
            LNSM_RLN       0.622175   0.020742          0.654555   
            LNSM_jaccard   0.519782   0.026209          0.531797   
indication  SKR            0.817447   0.021294          0.862328   
            KRR            0.810985   0.021653          0.852890   
            VKR            0.799529   0.031584          0.773931   
            Naive          0.785978   0.005667          0.803743   
            LNSM_RLN       0.633787   0.030442          0.639484   
            LNSM_jaccard   0.672306   0.031095          0.736589   
target      SKR            0.803289   0.019200          0.885943   
            KRR            0.803609   0.016149          0.886464   
            VKR            0.766297   0.017672          0.790806   
            Naive          0.791917   0.004910          0.813104   
            LNSM_RLN       0.654694   0.039555          0.682017   
            LNSM_jaccard   0.585003   0.057271          0.632572   
transporter SKR            0.774263   0.013933          0.804246   
            KRR            0.775350   0.011070          0.804929   
            VKR            0.760398   0.017762          0.761437   
            Naive          0.786424   0.007305          0.805034   
            LNSM_RLN       0.723054   0.018057          0.734979   
            LNSM_jaccard   0.451377   0.034800          0.435351   
enzyme      SKR            0.776731   0.024420          0.800188   
            KRR            0.766223   0.027136          0.791564   
            VKR            0.777781   0.014600          0.784162   
            Naive          0.795753   0.012765          0.814066   
            LNSM_RLN       0.735148   0.040977          0.740353   
            LNSM_jaccard   0.501857   0.010007          0.475465   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.024795  0.753108  0.008690  0.081934   
            KRR                   0.028389  0.711942  0.015692  0.080135   
            VKR                   0.008558  0.760231  0.008005  0.038941   
            Naive                 0.005101  0.767493  0.012370  0.024404   
            LNSM_RLN              0.147834  0.684964  0.138341  0.066716   
            LNSM_jaccard          0.056602  0.581359  0.074172  0.024730   
Chem        SKR                   0.025752  0.728132  0.024448  0.071061   
            KRR                   0.023691  0.697571  0.025012  0

In [47]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_rare.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            0.800483          0.832402  0.771716  0.028767   
            KRR            0.802500          0.832841  0.772980  0.029520   
            VKR            0.783390          0.792441  0.756020  0.027371   
            Naive          0.796940          0.824556  0.768120  0.028821   
            LNSM_RLN       0.359796          0.405702  0.341474  0.018323   
            LNSM_jaccard   0.417071          0.484296  0.406426  0.010645   
Chem        SKR            0.834835          0.878129  0.782746  0.052089   
            KRR            0.808519          0.838999  0.775254  0.033265   
            VKR            0.799166          0.795581  0.756581  0.042585   
            Naive          0.791128          0.816394  0.765827  0.025301   
            LNSM_RLN       0.673508          0.728270  0.619170  0.054338   
            LNSM_jaccard   0.521018          0.533893  0.510114  0.010903   
DGI         SKR            0.820121          0.848577  0.779703  0.040418   
            KRR            0.796386          0.817449  0.770216  0.026170   
            VKR            0.775438          0.778167  0.751571  0.023867   
            Naive          0.790185          0.810818  0.765628  0.024557   
            LNSM_RLN       0.630589          0.648311  0.588869  0.041720   
            LNSM_jaccard   0.639102          0.683814  0.618553  0.020548   
indication  SKR            0.872658          0.884526  0.795474  0.077185   
            KRR            0.789998          0.817133  0.767830  0.022168   
            VKR            0.779900          0.778254  0.757142  0.022759   
            Naive          0.793659          0.814666  0.770672  0.022987   
            LNSM_RLN       0.682446          0.661945  0.603803  0.078643   
            LNSM_jaccard   0.710778          0.726486  0.649518  0.061260   
target      SKR            0.795248          0.818455  0.768428  0.026820   
            KRR            0.797217          0.818630  0.768766  0.028451   
            VKR            0.777324          0.781173  0.751416  0.025908   
            Naive          0.789075          0.809474  0.764326  0.024749   
            LNSM_RLN       0.660784          0.693812  0.612944  0.047840   
            LNSM_jaccard   0.541456          0.580684  0.520802  0.020654   
transporter SKR            0.809039          0.831097  0.777431  0.031608   
            KRR            0.807857          0.827640  0.776350  0.031507   
            VKR            0.787307          0.795286  0.758160  0.029147   
            Naive          0.805878          0.827300  0.774668  0.031210   
            LNSM_RLN       0.741160          0.735449  0.712760  0.028401   
            LNSM_jaccard   0.426324          0.416612  0.417487  0.008836   
enzyme      SKR            0.799238          0.819450  0.767902  0.031336   
            KRR            0.796314          0.814063  0.766354  0.029960   
            VKR            0.779501          0.781556  0.751041  0.028460   
            Naive          0.793378          0.810952  0.764411  0.028968   
            LNSM_RLN       0.755503          0.771617  0.720526  0.034976   
            LNSM_jaccard   0.488458          0.480603  0.479253  0.009206   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.795311    0.037091  
            KRR              0.795415    0.037426  
            VKR              0.759717    0.032724  
            Naive            0.789031    0.035525  
            LNSM_RLN         0.360783    0.044919  
            LNSM_jaccard     0.431501    0.052795  
Chem        SKR              0.805465    0.072664  
            KRR              0.795312    0.0

In [48]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')